In [ ]:
# Install required libraries
!pip install transformers sentence-transformers faiss-cpu gradio
!pip install scikit-learn pandas numpy
!pip install swarm  # If unavailable, will give fallback later
!pip install langchain

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")


In [ ]:
print(df)

In [ ]:
import pandas as pd
import random

# Define sample queries for each intent
data = pd.read_csv("/content/all_tickets_processed_improved_v3.csv")
df = pd.DataFrame(data)

# Generate duplicates with slight modifications to expand dataset size to ~600 rows
def augment_text(text):
    # simple augment by rephrasing with synonyms, you can expand later
    replacements = {
        "How do I": "What is the way to",
        "My": "The",
        "How to": "What is the method to",
        "Where can I": "Where do I",
        "Can I": "Is it possible to"
    }
    for k, v in replacements.items():
        if text.startswith(k):
            return text.replace(k, v)
    return text

augmented_rows = []
for i in range(40):  # multiply data by 40 (15*40 = 600)
    for _, row in df.iterrows():
        new_text = augment_text(row['Document']) # Use 'summary' column
        augmented_rows.append({'summary': new_text, 'category': row['Topic_group']})

df_augmented = pd.DataFrame(augmented_rows)

# Shuffle the dataset
df_augmented = df_augmented.sample(frac=1).reset_index(drop=True)
df_augmented.head()

In [ ]:
knowledge_base = [
    {
        "title": "Laptop Reset Policy",
        "content": "To reset your company laptop, press F11 during boot and follow the on-screen instructions. Ensure backup is taken before reset."
    },
    {
        "title": "Password Reset Guide",
        "content": "Employees can reset passwords via the internal portal under Account Settings > Password Reset. For two-factor issues, contact IT support."
    },
    {
        "title": "HR Leave Policy",
        "content": "Full-time employees are entitled to 20 paid leave days per year. Submit requests on the HR portal at least 3 days in advance."
    },
    {
        "title": "Software Installation Request",
        "content": "Employees must raise a ticket through ServiceNow to request software installation. Licensing approval from the manager is required."
    },
    {
        "title": "Email Configuration",
        "content": "To configure company email on mobile, use Exchange settings with domain 'corp.company.com'. VPN must be active."
    }
]

# Convert to DataFrame
kb_df = pd.DataFrame(knowledge_base)
kb_df.to_csv("knowledge_base.csv", index=False)

# View
kb_df.head()


In [ ]:
!pip install langchain faiss-cpu
!pip install sentence-transformers
!pip install huggingface_hub
!pip install langchain-community # Ensure community package is installed here as well

# For T5 generation
!pip install transformers

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline
import pandas as pd

In [ ]:
# Load the knowledge base
kb_df = pd.read_csv("knowledge_base.csv")

# Use LangChain DataFrameLoader
loader = DataFrameLoader(kb_df, page_content_column="content")
documents = loader.load()

# Split text (optional here since docs are short)
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

# Use SentenceTransformer for embedding
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embed_model)


In [ ]:
# Set up text2text-generation pipeline using T5
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256)

# Wrap it in LangChain
llm = HuggingFacePipeline(pipeline=qa_pipeline)

# Build RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# Example Query Test
query = "How do I reset my company laptop?"
result = rag_chain(query)

print("Answer:", result["result"])
print("\nRetrieved Docs:")
for doc in result['source_documents']:
    print("—", doc.page_content)


In [ ]:
!pip install transformers datasets scikit-learn

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import Dataset
import torch

data = pd.read_csv("/content/all_tickets_processed_improved_v3.csv")

df = pd.DataFrame(data)

# Optional: augment synthetic data to increase size
def augment_text(text):
    replacements = {
        "How do I": "What is the way to",
        "My": "The",
        "How to": "What is the method to",
        "Where can I": "Where do I",
        "Can I": "Is it possible to"
    }
    for k, v in replacements.items():
        if text.startswith(k):
            return text.replace(k, v)
    return text

augmented_rows = []
for _ in range(40):  # replicate and augment ~40 times to get bigger dataset
    for _, row in df.iterrows():
        augmented_rows.append({'summary': augment_text(row['Document']), 'category': row['Topic_group']})

df_aug = pd.DataFrame(augmented_rows).sample(frac=1).reset_index(drop=True)

# Encode labels
le = LabelEncoder()
df_aug['label'] = le.fit_transform(df_aug['category'])

# Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_aug['summary'].tolist(), df_aug['label'].tolist(), test_size=0.2, random_state=42)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Dataset class
class IntentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=64)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = IntentDataset(train_texts, train_labels)
val_dataset = IntentDataset(val_texts, val_labels)


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

print(classification_report(true_labels, predictions, target_names=le.classes_))


In [ ]:
model.save_pretrained("bert_intent_classifier")


In [ ]:
import random

good_responses = [
    "To reset your laptop, hold the power button for 10 seconds and restart.",
    "Please contact HR at hr@company.com for leave policy clarification.",
    "You can access your payslip via the Employee Portal under Payroll section.",
    "Follow the steps in the IT manual page 42 for VPN configuration.",
    "We’ve escalated the issue to the technical team. You’ll hear back soon."
]

bad_responses = [
    "idk maybe try restarting it lol",
    "Check somewhere else for info",
    "What even is VPN?",
    "You should probably read the manual or something.",
    "Leave stuff? Ask someone in HR I guess."
]

# Duplicate and label
synthetic_texts = good_responses * 20 + bad_responses * 20
synthetic_labels = [1]*len(good_responses)*20 + [0]*len(bad_responses)*20  # 1=Good, 0=Bad

# Shuffle
combined = list(zip(synthetic_texts, synthetic_labels))
random.shuffle(combined)
texts, labels = zip(*combined)


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    list(texts), list(labels), test_size=0.2, random_state=42)

class ResponseDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=64)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = ResponseDataset(train_texts, train_labels)
val_dataset = ResponseDataset(val_texts, val_labels)


In [ ]:
refiner_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
refiner_model.to(device)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

optimizer = AdamW(refiner_model.parameters(), lr=3e-5)

# Training loop
for epoch in range(3):
    refiner_model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = refiner_model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"[Refiner] Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
refiner_model.eval()
refiner_preds, refiner_true = [], []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = refiner_model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        refiner_preds.extend(preds.cpu().numpy())
        refiner_true.extend(batch['labels'].cpu().numpy())

print(classification_report(refiner_true, refiner_preds, target_names=["Needs Improvement", "Good"]))


In [ ]:
class RouterAgent:
    def __init__(self, model, tokenizer, label_map):
        self.model = model
        self.tokenizer = tokenizer
        self.label_map = label_map
        self.model.eval()

    def route(self, query):
        inputs = self.tokenizer(query, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = self.model(**inputs).logits
        predicted_class = torch.argmax(logits, dim=1).item()
        return self.label_map[predicted_class]


In [ ]:
label_map = {0: "IT", 1: "HR", 2: "Product"}  # based on your classifier's training
router_agent = RouterAgent(model, tokenizer, label_map)
router_agent.route("How can I access my salary slip?")


In [ ]:
class ResponseRefinerAgent:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()

    def refine(self, response):
        inputs = self.tokenizer(response, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = self.model(**inputs).logits
        prediction = torch.argmax(logits, dim=1).item()
        return "Acceptable" if prediction == 1 else "Needs Improvement"


In [ ]:
refiner_agent = ResponseRefinerAgent(refiner_model, tokenizer)
refiner_agent.refine("Please turn off your laptop and contact IT.")


In [ ]:
!pip install langchain faiss-cpu sentence-transformers transformers


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer

# Sample documents
kb_texts = [
    "To reset your laptop, press and hold the power button for 10 seconds.",
    "To apply for leave, log in to the HR portal and click on 'Apply Leave'.",
    "Access the company VPN using your employee credentials on the VPN client.",
    "Payslips are available in the Payroll section of the Employee Portal.",
    "For hardware issues, contact IT at support@company.com."
]

documents = [Document(page_content=text) for text in kb_texts]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", k=3)


In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline

t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

t5_pipe = pipeline("text2text-generation", model=t5_model, tokenizer=t5_tokenizer)
generator = HuggingFacePipeline(pipeline=t5_pipe)


In [ ]:
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=generator,
    retriever=retriever,
    return_source_documents=False
)

def retrieve_and_generate(query):
    return rag_chain.run(query)


In [ ]:
response = retrieve_and_generate("How do I access my payslip?")
print(response)


In [ ]:
class HelpdeskOrchestrator:
    def __init__(self, router_agent, rag_chain, refiner_agent):
        self.router = router_agent
        self.rag = rag_chain
        self.refiner = refiner_agent

    def handle_query(self, query):
        print(f"\n🧭 [Router] Classifying intent...")
        intent = self.router.route(query)
        print(f"✅ Detected intent: {intent}\n")

        print("📚 [Knowledge Specialist] Generating response...")
        response = self.rag.run(query)
        print(f"💬 Response:\n{response}\n")

        print("🧪 [Response Refiner] Evaluating quality...")
        evaluation = self.refiner.refine(response)
        print(f"🔎 Response Quality: {evaluation}\n")

        return {
            "intent": intent,
            "response": response,
            "evaluation": evaluation
        }


In [ ]:
orchestrator = HelpdeskOrchestrator(router_agent, rag_chain, refiner_agent)

query = "How do I apply for medical leave?"
output = orchestrator.handle_query(query)

print("Final Output:", output)


In [ ]:
!pip install gradio

import gradio as gr

def interactive_helpdesk(query):
    result = orchestrator.handle_query(query)
    return f"""🔹 Intent: {result['intent']}

💬 Response:
{result['response']}

🧪 Evaluation: {result['evaluation']}
"""

gr.Interface(fn=interactive_helpdesk, inputs="text", outputs="text", title="🧠 Swarm-Powered Enterprise Helpdesk").launch()
